In [4]:
import os
import nltk
import pandas as pd
import time
import string
import re
import tkinter
import sys
from pympler import asizeof
from tqdm import tqdm
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from IPython.display import clear_output

# !pip install pympler

### 0.0 List the Files in the specific directory (Section 0.0)

In [5]:
def listFile(d):    
    path = [os.path.abspath(os.path.join(d,i)) for i in os.listdir(d)]
    return path

### 0.1 Read the file and return the content (Section 0.1)

In [6]:
def readFile(d):
    file = open(d,"r",encoding='utf-8')
    content = file.read()
    return content

### 1. Create token

In [2]:
#unoptimized (input - content, document number)
def createToken(content, dnum):
    tokens = list()
    tokenizer = WhitespaceTokenizer()
    for t in tokenizer.tokenize(content):
        tokens = tokens + [[t,dnum]]
    return tokens

In [8]:
#optimized (input takes only content)
def createTokenOptimized(content):
    tokens = list()
    tokenizer = WhitespaceTokenizer()
    for token in tokenizer.tokenize(content):
        tokens.append(token)
    return tokens

### 2. Linguistic (lower, stemming)

In [9]:
def linguisticToken(token_list):
    stemmer = PorterStemmer() #stem
    #lemmer = WordNetLemmatizer()
    tokens = list()
    regex = "[!@#$%^&*()-_=+'`~ \":;|/.,?[]{}<>]"
    for [t,d] in token_list:
        token = t.translate(str.maketrans('', '', regex)) #remove punctuations
        if token == '': #if the token is only punctuation
            continue
        token = token.lower()
        token = stemmer.stem(token)
        #token = lemmer.lemmatize(token)
        tokens += [[token,d]]
    return tokens

In [10]:
#optimized (input takes only content)
def linguisticTokenOptimized(token_list):
    stemmer = PorterStemmer() #stem
    #lemmer = WordNetLemmatizer()
    tokens = list()
    regex = "[!@#$%^&*()-_=+'`~ \":;|/.,?[]{}<>]"
    for t in token_list:
        token = t.translate(str.maketrans('', '', regex)) #remove punctuations
        if token == '': #if the token is only punctuation
            continue
        token = token.lower()
        token = stemmer.stem(token)
        #token = lemmer.lemmatize(token)
        tokens.append(token)
    return tokens

### 3. Sorting

In [12]:
#unoptimized (sort by term, document id)
def sortToken(token_list):
    token_list.sort(key=lambda e: (e[0],e[1]))
    return token_list

In [13]:
#sort by terms only
def sortTokenFirstOpt(token_list):
    token_list.sort(key=lambda e: e[0])
    return token_list

In [14]:
#sort only by terms and append docID
def sortTokenOptimized(listStr):
    #print(listStr)
    newList=[]
    for doc in listStr:
        for term in doc[0]:
            newList.append([term, doc[1]])
    sortedToken = sorted(newList, key = lambda x: x[0])
    return sortedToken

### 4. Transform into posting

In [16]:
def transformPosting(sorted_list):
    postDictionary = {}
    for term,docId in sorted_list: #add terms into posting
        postDictionary.setdefault(term,[]).append(docId)
    for key in postDictionary:
        post = list(dict.fromkeys(postDictionary[key]))
        post.sort(key=str)
        postDictionary[key] = [len(post),post]
    return postDictionary

In [15]:
def transformPostingOptimized(sorted_list):
    postDictionary = {}
    for term,docId in sorted_list: #add terms into posting
        postDictionary.setdefault(term,[]).append(docId)
    for key in postDictionary:
        post = list(dict.fromkeys(postDictionary[key]))
        post.sort(key=int)
        postDictionary[key] = [len(post),post]
    return postDictionary

### 5. Merge the Postings (Intersecting)

In [18]:
#implementation of the algorithm from lecture (for AND operations)
def mergePostings(postingList):
    result = []
    posting1 = postingList[0]
    
    for i in range(1,len(postingList),1):
        merged = []
        posting2 = postingList[i]
        p = 0
        q = 0
        while p < len(posting1) and q < len(posting2):
            if int(posting1[p]) == int(posting2[q]):
                merged.append(posting1[p])
                p += 1
                q += 1
            elif int(posting1[p]) < int(posting2[q]):
                p += 1
            else:
                q += 1
        posting1 = merged
    return posting1

### Merge Postings (for OR operations)

In [3]:
#for OR operations
def mergeOrPostingsOR(postingList):
    posting1 = postingList[0]
    
    for i in range(1,len(postingList),1):
        merged = []
        posting2 = postingList[i]
        p = 0
        q = 0
        while p < len(posting1) and q < len(posting2):
            if int(posting1[p]) == int(posting2[q]):
                merged.append(posting1[p])
                p += 1
                q += 1
            elif int(posting1[p]) < int(posting2[q]):
                merged.append(posting1[p])
                p += 1
            else:
                merged.append(posting2[q])
                q += 1
        if p < len(posting1):
            merged += posting1[p:]
        elif q < len(posting2):
            merged += posting2[q:]
        posting1 = merged
    return posting1  

### 6. Create Index

In [24]:
#create inverted index for the directory
def non_optimized(directory):
    start_time = time.time()
    
    print("Reading files from ", directory)
    files = listFile(directory) #list dir
    tokens = list()
    num=0
    did={}
    for docs in tqdm(files):
        try:
            file_content = readFile(docs) #read content
            token = createToken(file_content,num) #create token
            token = linguisticToken(token) #stemming
            tokens += token
            did[num]=docs
            num+=1
        except:
            print(docs)
    
    print("Sorting the tokens ...")
    tokens = sortToken(tokens) #token from all files

    print("Transforming into postings ...")
    posting = transformPosting(tokens) #create posting from these files

    
    end_time = time.time()
    time_to_index = end_time - start_time
    print("Finished indexing.\nTime taken to index: " , round(time_to_index,3))
    return posting, time_to_index

In [22]:
#create inverted index for the directory
def first_optimized(directory):
    start_time = time.time()
    
    print("Reading files from ", directory)
    files = listFile(directory) #list dir
    tokens = list()
    num = 0
    did = {}
    for docs in tqdm(files):
        try:
            file_content = readFile(docs) #read content
            token = createToken(file_content,num) #create token
            token = linguisticToken(token) #stemming
            tokens += token
            did[num] = docs
            num +=1
        except:
            print(docs)
    
    #print(tokens)
    print("Sorting the tokens ...")
    tokens = sortTokenFirstOpt(tokens) #token from all files
    
    print("Transforming into postings ...")
    posting = transformPosting(tokens) #create posting from these files

    end_time = time.time()
    time_to_index = end_time - start_time
    print("Finished indexing.\nTime taken to index: " , round(time_to_index,3))
    return posting, did, time_to_index

In [25]:
#optimized inverted index
def optimized(directory):
    start_time = time.time()
    
    print("Reading files from " , directory)
    files = listFile(directory) #list dir
    tokens = list()
    num = 0
    did = {}
    for docs in tqdm(files):
        #print(docs,num)
        try:
            file_content = readFile(docs) #read content
            token = createTokenOptimized(file_content) #create token
            token = linguisticTokenOptimized(token) #stemming
            token_id = [token,num]
            tokens += [token_id]
            did[num] = docs
            num +=1
        except:
            print(docs)
            
    print("Sorting the tokens ...")
    
    tokens = sortTokenOptimized(tokens) #token from all files
    
    print("Transforming into postings ...")
    posting = transformPostingOptimized(tokens)
    end_time = time.time()
    time_to_index = end_time - start_time
    print("Finished indexing.\nTime taken to index: " , round(time_to_index,3))
    return posting, did, time_to_index

### Process the Query (transform into token, search, merge and return)

In [26]:
def processQuery(query):
    q_token = createTokenOptimized(query)
    q_token = linguisticTokenOptimized(q_token)
    posting_list = []
    
    for token in q_token:
        try:
            posting_list.append(posting[token][1])
        except:
            print(token)
            posting_list.append([])
            break
    result = mergePostings(posting_list)
    return result

### Run the indexer (Normal)

In [27]:
posting, time_to_index = non_optimized("datafull2")
print("Size of index: ", asizeof.asizeof(posting), " bytes")

Reading files from  datafull2


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 14.53it/s]


Sorting the tokens ...
Transforming into postings ...
Finished indexing.
Time taken to index:  0.084
Size of index:  171128  bytes


In [30]:
newlist=[]
for key in posting:
    newlist.append([key,posting[key][0]])
newlist.sort(key=lambda x:x[1],reverse=True)
print(newlist)
posting['gargalesthesia']

[['and', 54], ['in', 54], ['of', 54], ['the', 54], ['a', 53], ['is', 53], ['to', 53], ['as', 52], ['for', 51], ['with', 51], ['by', 50], ['from', 50], ['also', 49], ['an', 49], ['but', 49], ['it', 49], ['on', 48], ['that', 48], ['are', 47], ['at', 47], ['be', 47], ['ha', 47], ['which', 47], ['use', 46], ['not', 45], ['or', 45], ['wa', 45], ['have', 43], ['onli', 43], ['thi', 43], ['more', 42], ['other', 42], ['they', 42], ['such', 41], ['were', 41], ['can', 40], ['all', 39], ['first', 39], ['known', 39], ['most', 39], ['one', 39], ['when', 39], ['into', 38], ['some', 38], ['their', 38], ['two', 38], ['while', 38], ['had', 37], ['may', 37], ['there', 37], ['these', 37], ['been', 36], ['follow', 36], ['form', 36], ['mani', 36], ['then', 36], ['time', 36], ['between', 35], ['both', 35], ['call', 35], ['develop', 35], ['number', 35], ['part', 35], ['than', 35], ['under', 35], ['where', 35], ['about', 34], ['howev', 34], ['includ', 34], ['larg', 34], ['later', 34], ['result', 34], ['after',

[1, ['C:\\Users\\melvi\\Desktop\\data_sample\\Aaaaa.txt']]

### Run the indexer (1st optimized)

In [146]:
posting, docId, time_to_index = first_optimized("ds")
print("Size of index: ", asizeof.asizeof(posting),"size of dictionary", asizeof.asizeof(docId), " bytes")

Reading files from  ds


100%|████████████████████████████████████████████████████████████████████████████████| 253/253 [00:08<00:00, 29.99it/s]


Sorting the tokens ...
Transforming into postings ...
Finished indexing.
Time taken to index:  8.863
Size of index:  7838496 size of dictionary 86456  bytes


### Run the indexer (2nd Optimized)

In [35]:
posting, docId, time_to_index = optimized("datafull-lean")
print("Size of index: ", asizeof.asizeof(posting)/1000000,"size of dictionary", asizeof.asizeof(docId)/1000000, "MB")

Reading files from  datafull-lean


100%|████████████████████████████████████| 47938/47938 [23:58<00:00, 33.33it/s]


Sorting the tokens ...
Transforming into postings ...
Finished indexing.
Time taken to index:  1648.891
Size of index:  628.085488 size of dictionary 15.494416 MB


In [38]:
asizeof.asizeof(posting)/1000000

628.085488

In [45]:
def posting_compression(data):
    new_data = {}
    for i in data:
        value = []
        for docID in data[i]:
            if isinstance(docID, list):
                new_docID = []
                new_docID.append(docID[0])
                for index_of_docID in range(len(docID)-1):
                    new_docID.append(docID[index_of_docID + 1] - docID[index_of_docID])
            else:
                value.append(docID)
        value.append(new_docID)
        new_data[i] = value
    return new_data

# # {term: [freq,[list]]}
# def posting_compression2(data):
#     new_data = {}
    
    
#     return new_data

def dict_as_a_str_compression(posting):
    term_ptr_list=[0]
    newDictString = ''
    dict_output = {}
    k = 0
    for key in posting:
        newDictString += key
        term_ptr_list.append(len(key))
    #     print(key)
    #     print(posting[key])
        dict_output[k] = posting[key]
        k += len(key)
#         print(newDictString)
    return [dict_output,newDictString]

In [78]:
dictTest={'melvin':[9,[0,3,6,7,8,123,164,244,734]],'is':[14,[31,46,76,97,142,162,172,242,262,331,361,363,371,380]]}
dictTestCompressed=posting_compression(dictTest)
print(asizeof.asizeof(dictTest))
print(asizeof.asizeof(dictTestCompressed))


1616
1560


In [48]:
# dict_output = dict_as_a_str_compression(posting)[0]
# dict_output_size=asizeof.asizeof(dict_output)/1000000
# print('dict', dict_output_size)

post_comp = posting_compression(posting)
post_comp_size=asizeof.asizeof(post_comp)/1000000
print('posting', post_comp_size)



posting 713.849752


In [75]:
print('posting', asizeof.asizeof(posting)/1000000)

posting 628.085488


In [76]:
len(posting)

1477855

In [73]:
type(posting['0'][1][0])

int

In [52]:
newList=[]
for key in posting:
    newList.append([len(posting[key]),len(post_comp[key])])
newList

listA=[2,3]
listB=[1]

[[2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 [2, 2],
 

In [47]:
dict_string_output_size

55.763536

In [44]:
dict_string_output = dict_as_a_str_compression(posting)[1]
dict_string_output_size=asizeof.asizeof(dict_string_output)/1000000

In [69]:
import pickle

pickle.dump(posting, open("backup/posting.p","wb"))
pickle.dump(docId,open("backup/docId.p","wb"))
pickle.dump(time_to_index,open("backup/time_to_index.p","wb"))

posting=pickle.load(open("backup/posting.p","rb"))
docId=pickle.load(open("backup/docId.p","rb"))
time_to_index=pickle.load(open("backup/time_to_index.p","rb"))

In [3]:
posting=pickle.load(open("backup/posting.p","rb"))
asizeof.asizeof(posting)

1352448704

In [72]:
posting

{'0': [1979,
  [1,
   9,
   10,
   12,
   15,
   16,
   81,
   117,
   209,
   211,
   214,
   224,
   226,
   229,
   230,
   232,
   233,
   234,
   235,
   236,
   237,
   260,
   265,
   268,
   320,
   332,
   351,
   369,
   396,
   408,
   415,
   457,
   518,
   589,
   613,
   660,
   784,
   797,
   801,
   837,
   981,
   989,
   1069,
   1114,
   1247,
   1286,
   1381,
   1413,
   1416,
   1418,
   1426,
   1427,
   1430,
   1431,
   1441,
   1447,
   1448,
   1449,
   1450,
   1473,
   1482,
   1484,
   1485,
   1486,
   1490,
   1497,
   1499,
   1501,
   1505,
   1520,
   1523,
   1541,
   1556,
   1583,
   1620,
   1669,
   1674,
   1698,
   1807,
   1815,
   1816,
   1865,
   1923,
   1941,
   1945,
   1974,
   1989,
   1999,
   2004,
   2006,
   2007,
   2022,
   2023,
   2112,
   2139,
   2153,
   2166,
   2214,
   2311,
   2314,
   2316,
   2335,
   2359,
   2429,
   2481,
   2482,
   2483,
   2486,
   2489,
   2532,
   2536,
   2575,
   2609,
   2640,
   2641,
   

## Run the Search Engine (without GUI)

In [36]:
start_time1 = time.time()
query1="in of the to as from also it and is an develop god Gargalesthesia"
for i in range(10000):
    result = processQuery(query1)
    
print( len(result), "documents found for ", query1)
print("Time taken to search: ", time.time()-start_time1)

start_time2 = time.time()
query2="Gargalesthesia god develop an is and it also from as to the of in"
for i in range(10000):
    result = processQuery(query2)
    
print( len(result), "documents found for ", query2)
print("Time taken to search: ", time.time()-start_time2)

1 documents found for  in of the to as from also it and is an develop god Gargalesthesia
Time taken to search:  2.791602611541748
1 documents found for  Gargalesthesia god develop an is and it also from as to the of in
Time taken to search:  1.904170036315918


In [79]:
while True:
    query = input("Enter a query (type q to exit) : ")
    if query == "q":
        break
    clear_output()
    start_time = time.time()
    result = processQuery(query)
    
    print( len(result), "documents found for ", query)
    print("Time taken to search: ", time.time()-start_time)
    
    for doc in result:
        print(doc)

2 documents found for  infect
Time taken to search:  0.0
C:\Users\melvi\Desktop\data_sample\Herrerasaurus.txt
C:\Users\melvi\Desktop\data_sample\Yahya_Jammeh.txt
Enter a query (type q to exit) : q


## Run the Search Engine (with GUI)

In [32]:
window = tkinter.Tk()

def processBtn():
    start_time = time.time()
    query = inputEntry.get()
    result = processQuery(query)
    toShow = str(len(result)) + " documents found for " + query + "\n"
    toShow += "Time taken to search: " + str(time.time()-start_time) +"\n"
    rs.config(text = toShow)
    rsFiles = ""
    for num, doc in enumerate(result):
        rsFiles += str(num) + ". " + os.path.basename(docId[int(doc)]) + "\n"
    rsdetail.config(text = rsFiles)  

window.title("NTU Search Engine")
window.geometry("600x1024")
label = tkinter.Label(window, text = "Welcome to our Search Engine", width = 60, fg="red")
label.grid(row = 0)
inputEntry = tkinter.Entry(window,text = "Type here to search", width= 60)
inputEntry.grid(row=1, column = 0)
button_widget = tkinter.Button(window,text="Search", command = processBtn)
button_widget.grid(row=1, column = 1)
rs = tkinter.Label(window, text="", fg="red")
rs.grid(row=3, column = 0, columnspan=2)

rsdetail = tkinter.Message(window, bg="white")
rsdetail.grid(row=4, column = 0, columnspan=2, sticky = "w")

window.mainloop()

### Test the time comparison between Normal and Optimized (average of 10 runs with data sample of 100 files)

In [21]:
time_normal = 0
time_opt = 0
for i in range(10):
    print("Running ", i, " iteration for optimized one")
    pO, dO, time_to_index_opt = non_optimized1("data_sample")
    time_opt += time_to_index_opt
    clear_output()
    print("Running ", i, " iteration for normal one")
    p, d, time_to_index_normal = non_optimized("data_sample")
    time_normal += time_to_index_normal
    clear_output()
print("Normal: " , time_normal/10, "seconds average")
print("Optimized: ", time_opt/10, "seconds average")
print(((time_normal-time_opt)/time_opt)*100, "% faster")

Normal:  2.983490324020386 seconds average
Optimized:  2.357524847984314 seconds average
26.551808205596345 % faster


In [43]:
a = "hello"
b = "hello"
a==b

True